In [1]:
# Importing modules
import numpy as np
import os
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#Load Data
np.random.seed(100) # for reproducibility
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

#cifar-10 has images of airplane, automobile, bird, cat,
# deer, dog, frog, horse, ship and truck ( 10 unique labels)
# For each image. width = 32, height =32, Number of channels(RGB) = 3

In [3]:
#Preprocess the data
#Flatten the data, MLP doesn't use the 2D structure of the data. 3072 = 3*32*32
X_train = X_train.reshape(50000, 3072) # 50,000 images for training
X_test = X_test.reshape(10000, 3072) # 10,000 images for test

# Making data Float type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Gaussian Normalization( Z- score)
X_train = (X_train- np.mean(X_train))/np.std(X_train)
X_test = (X_test- np.mean(X_test))/np.std(X_test)

# Display number of training and test instances
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')



50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices (ie one-hot vectors)
nb_classes = 10 #One class per digit
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [5]:
#Define the model achitecture
model = Sequential()
model.add(Dense(512, input_shape=(3072,))) # 3*32*32 = 3072
model.add(Activation('relu'))
model.add(Dropout(0.2)) # Regularization
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10)) #Last layer with one output per class
model.add(Activation('softmax')) #We want a score simlar to a probability for each class

In [6]:
# Compile the model
# Use rmsprop as an optimizer
rms = RMSprop()
#The function to optimize is the cross entropy between the true label and the output (softmax) of the model
model.compile(loss='categorical_crossentropy', optimizer=rms, metrics=["accuracy"])

In [10]:
#Make the model learn ( Fit the model)
batch_size = 1000 #Number of images used in each optimization step
nb_epoch = 20 #Number of times the whole data is used to learn
model.fit(X_train, Y_train,batch_size=batch_size, nb_epoch=nb_epoch,validation_data=(X_test, Y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 4s - loss: 1.0959 - acc: 0.6100 - val_loss: 1.3467 - val_acc: 0.5363
Epoch 2/20
50000/50000 [==============================] - 4s - loss: 1.0729 - acc: 0.6155 - val_loss: 1.3423 - val_acc: 0.5406
Epoch 3/20
50000/50000 [==============================] - 3s - loss: 1.0498 - acc: 0.6246 - val_loss: 1.3465 - val_acc: 0.5402
Epoch 4/20
50000/50000 [==============================] - 3s - loss: 1.0422 - acc: 0.6272 - val_loss: 1.3693 - val_acc: 0.5402
Epoch 5/20
50000/50000 [==============================] - 3s - loss: 1.0285 - acc: 0.6313 - val_loss: 1.3733 - val_acc: 0.5354
Epoch 6/20
50000/50000 [==============================] - 3s - loss: 1.0135 - acc: 0.6394 - val_loss: 1.3372 - val_acc: 0.5416
Epoch 7/20
50000/50000 [==============================] - 3s - loss: 0.9978 - acc: 0.6418 - val_loss: 1.3572 - val_acc: 0.5382
Epoch 8/20
50000/50000 [==============================] - 3s 

In [11]:
#Evaluate how the model does on the test set
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 1.39958637295
Test accuracy: 0.5539


In [12]:
#Predict digit(0-9) for test Data
model.predict_classes(X_test)

 9920/10000 [============================>.] - ETA: 0s

array([3, 9, 8, ..., 3, 6, 4], dtype=int64)

In [13]:
#Saving the model
model.save('model.h5')
jsonModel = model.to_json()
model.save_weights('modelWeight.h5')

In [14]:
#Summary of the model
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               61560     
_________________________________________________________________
activation_2 (Activation)    (None, 120)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1210      
__________

In [15]:
from keras.models import model_from_json
model_from_json(jsonModel)

In [16]:
modelWt = model.load_weights('modelWeight.h5')

In [17]:
model.get_config()

[{'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'batch_input_shape': (None, 3072),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'VarianceScaling',
    'config': {'distribution': 'uniform',
     'mode': 'fan_avg',
     'scale': 1.0,
     'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_1',
   'trainable': True,
   'units': 512,
   'use_bias': True}},
 {'class_name': 'Activation',
  'config': {'activation': 'relu', 'name': 'activation_1', 'trainable': True}},
 {'class_name': 'Dropout',
  'config': {'name': 'dropout_1', 'rate': 0.2, 'trainable': True}},
 {'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': No

In [18]:
model.get_weights()

[array([[-0.04820345, -0.04580304,  0.03334169, ...,  0.03378518,
          0.06373712,  0.06177348],
        [-0.02929454,  0.00496516, -0.09331881, ..., -0.00633003,
          0.0570285 , -0.04104728],
        [-0.03909263, -0.06397794, -0.03683902, ...,  0.0125256 ,
          0.00996903, -0.04296384],
        ..., 
        [ 0.07559144, -0.0623954 ,  0.01372101, ...,  0.01776433,
         -0.00949753, -0.05391226],
        [-0.06938065, -0.01544604, -0.02301907, ..., -0.00404552,
          0.04807976, -0.069023  ],
        [ 0.02173178, -0.05709683,  0.03162104, ...,  0.04941871,
          0.04069708, -0.04208992]], dtype=float32),
 array([ -1.57667950e-01,  -1.54690564e-01,  -3.35931689e-01,
         -4.66868818e-01,   2.70230602e-02,  -1.48925766e-01,
         -1.62333101e-01,   8.78315568e-02,  -8.63119587e-02,
         -1.98889345e-01,  -6.16785586e-01,  -2.85586715e-01,
         -2.21329808e-01,  -4.39982504e-01,  -4.17092532e-01,
         -2.08222091e-01,   2.09666997e-01,  -2